In [1]:
# To imlpement the movie rating recommendation system
# output : ratings from 1-5

**Importing libraries:**

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from io import StringIO

**Importing the dataset:**

In [3]:
movies = pd.read_csv("/content/movies.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

In [4]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
users = pd.read_csv("/content/users.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

In [6]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
ratings = pd.read_csv("/content/ratings.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

In [8]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Preparing the Training and test set:

In [9]:
# using several train-test : for k fold testing
training_set = pd.read_csv("/content/user.base", delimiter = "\t", names = ["user", "movie", "rating","timestamp"])

In [10]:
training_set.head()

,user,movie,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [11]:
training_set = np.array(training_set, dtype= "int")

In [12]:
test_set = pd.read_csv("/content/user.test", delimiter = "\t", names = ["user", "movie", "rating","timestamp"])
test_set = np.array(test_set, dtype = "int")

**Getting the number of users and movies:**

In [13]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print("nb_users : {} \nnb_movies : {}".format(nb_users, nb_movies))

nb_users : 943 
nb_movies : 1682


In [14]:
# converting into matirx:
def convert(data):
  hors_lis = []
  for ith_user in range(1, nb_users + 1):
    movies = data[:,1][data[:,0] == ith_user]
    ratings = data[:,2][data[:,0] == ith_user]
    all_ratings = np.zeros(nb_movies)
    all_ratings[movies-1] = ratings
    all_ratings = list(all_ratings)
    hors_lis.append(all_ratings)
  return hors_lis

In [15]:
# converison:
training_set = convert(training_set)
test_set = convert(test_set)

In [16]:
print("training_set :")
print(training_set[0])
print("\n\ntest_set :")
print(test_set[0])

training_set :
[5.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 5.0,

Converting the data into Toch tensors:

In [17]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Architecture of Autoencoder:

In [18]:
# Taking parent class --> "nn" from torch.nn
# Create a model class of autoencoder (through inheritance)
# SAE : stacked auto-encoder

class SAE(nn.Module):
  def __init__(self,):
    super(SAE, self).__init__()

    # fc1 : 1st full connection layer of neurons:
    self.fc1 = nn.Linear(nb_movies, 20,)

    # fc2 : 2nd layer
    self.fc2 = nn.Linear(20, 10)

    # 3rd layer : should have output neuron to 1st layer's input neurons:
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()

  # encoding and decoding action of autoencoder:
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

  def predict(self,user_input):
    user_input = torch.FloatTensor(user_input).view(1,-1)
    user_input = Variable(user_input)
    predicted_ratings = self(user_input)
    return predicted_ratings

In [19]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

**Training the SAE:**

In [20]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
  train_loss = 0
  s = 0.
  total_rmse = 0
  total_predictions = 0

  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    # Copying the input into target vector:
    # "taregt" contains at leats 1 rating that is not zero:
    target = input.clone()

    if torch.sum(target.data > 0) > 0:
      output = sae(input) # Getting the output of vector predictive ratings
      target.require_grad = False # To make gradient computed only wrt input and not target vector

      # Logic --> rating = 0, not be counted in computation:
      output[target==0] = 0

      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # avg of the error by the movies(non-zero) that were only rated
      rmse = np.sqrt(loss.data*mean_corrector)
      train_loss += rmse

      mae = torch.abs(output - target).sum() / torch.sum(target > 0).float()

      total_predictions +=1
      loss.backward() # Decides direction to which the weight be updated
      s+=1.

      # Decides the intensity of the object / the amount by which the weights be updated
      optimizer.step()
  print("epoch: {}    loss : {}     MAE : {}".format(epoch, train_loss/s, mae))

epoch: 1    loss : 1.771846890449524     MAE : 1.0117127895355225
epoch: 2    loss : 1.0968608856201172     MAE : 0.9936851263046265
epoch: 3    loss : 1.0535191297531128     MAE : 0.9922523498535156
epoch: 4    loss : 1.0385127067565918     MAE : 0.9907214045524597
epoch: 5    loss : 1.0305967330932617     MAE : 0.9906579256057739
epoch: 6    loss : 1.0267972946166992     MAE : 0.9919956922531128
epoch: 7    loss : 1.0237494707107544     MAE : 0.9854851961135864
epoch: 8    loss : 1.0219850540161133     MAE : 0.9897849559783936
epoch: 9    loss : 1.0209555625915527     MAE : 0.988010585308075
epoch: 10    loss : 1.0194648504257202     MAE : 0.9898484349250793
epoch: 11    loss : 1.0189646482467651     MAE : 0.9835308790206909
epoch: 12    loss : 1.0183807611465454     MAE : 0.9890475273132324
epoch: 13    loss : 1.0179675817489624     MAE : 0.9797742962837219
epoch: 14    loss : 1.0173985958099365     MAE : 0.9869457483291626
epoch: 15    loss : 1.017095685005188     MAE : 0.984079897

**Test loss:**

In [21]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)

  if torch.sum(target.data > 0) >0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data * mean_corrector)
    s += 1.
print("test loss : {}".format(test_loss/s))

test loss : 0.9615423083305359


**Predicting the Values of test_set:**

In [22]:
user_input = test_set[0]
predicted_ratings = sae.predict(user_input)
print("predicted ratings:")
print(predicted_ratings)

predicted ratings:
tensor([[3.9302, 3.5042, 3.0618,  ..., 2.0217, 3.2383, 3.2623]],
       grad_fn=<AddmmBackward0>)
